In [1]:
from llama_index.llms.ollama import Ollama
from llama_parse import LlamaParse
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, PromptTemplate
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from pydantic import BaseModel, Field
from llama_index.core.output_parsers import PydanticOutputParser
from llama_index.core.query_pipeline import QueryPipeline
from prompts import context, code_parser_template
from code_reader import code_reader
from dotenv import load_dotenv
import os
import ast
from typing import List

[nltk_data] Downloading package punkt_tab to
[nltk_data]     d:\Workspace\personal\ai_agent\venv\Lib\site-
[nltk_data]     packages\llama_index\core\_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
LLAMAPARSE_API_KEY = os.environ.get("LLAMAPARSE_API_KEY")

In [3]:
llm = Ollama(model="mistral", request_timeout=30.0)

parser = LlamaParse(result_type="markdown", api_key=LLAMAPARSE_API_KEY)
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader("./data", file_extractor=file_extractor).load_data()

embed_model = resolve_embed_model("local:BAAI/bge-m3")
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
query_engine = vector_index.as_query_engine(llm=llm)

result = query_engine.query("what are some of the routes in the api?")
print(result)

Failed to load file d:\Workspace\personal\ai_agent\data\readme.pdf with error: The event loop is already running. Add `import nest_asyncio; nest_asyncio.apply()` to your code to fix this issue.. Skipping...


d:\Workspace\personal\ai_agent\venv\Lib\site-packages\llama_index\core\readers\file\base.py:553: RuntimeWarning: coroutine 'LlamaParse.aload_data' was never awaited
  return []
d:\Workspace\personal\ai_agent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 In the provided API, there are several routes defined:

1. `/items` (HTTP methods: POST, GET) - Used for creating new items and retrieving all existing items.
2. `/items/<int:item_id>` (HTTP methods: GET, PUT, DELETE) - Used for retrieving a specific item, updating an existing item with the given ID, or deleting an item with the given ID.
   Note that the `<int:item_id>` part of the route indicates that the variable `item_id` should be an integer and will be automatically parsed from the requested URL.


In [4]:
tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="api_documentation",
            description="this gives documentation about code for an API. Use this for reading docs for the API",
        ),
    ),
    code_reader,
]

code_llm = Ollama(model="codellama")
agent = ReActAgent.from_tools(tools, llm=code_llm, verbose=True, context=context)

class CodeOutput(BaseModel):
    code: str
    description: str
    filename: str


parser = PydanticOutputParser(CodeOutput)
json_prompt_str = parser.format(code_parser_template)
print(json_prompt_str)
json_prompt_tmpl = PromptTemplate(json_prompt_str)
print("##########################\n")
print(json_prompt_tmpl)
output_pipeline = QueryPipeline(chain=[json_prompt_tmpl, llm])

Parse the response from a previous LLM into a description and a string of valid code, 
                            also come up with a valid filename this could be saved as that doesnt contain special characters. 
                            Here is the response: {response}. You should parse this in the following JSON Format: 


Here's a JSON schema to follow:
{{"properties": {{"code": {{"title": "Code", "type": "string"}}, "description": {{"title": "Description", "type": "string"}}, "filename": {{"title": "Filename", "type": "string"}}}}, "required": ["code", "description", "filename"], "title": "CodeOutput", "type": "object"}}

Output a valid JSON object but do not repeat the schema.

##########################

metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['response'] kwargs={} output_parser=None template_var_mappings=None function_mappings=None template='Parse the response from a previous LLM into a description and a string of valid code, \n                

In [5]:
while (prompt := input("Enter a prompt (q to quit): ")) != "q":     #   read the contents of test.py and write a python script that calls the post end point to make a new item.
    retries = 0

    while retries < 3:
        try:
            result = agent.query(prompt)
            next_result = output_pipeline.run(response=result)
            cleaned_json = ast.literal_eval(str(next_result).replace("assistant:", ""))
            break
        except Exception as e:
            retries += 1
            print(f"Error occured, retry #{retries}:", e)

    if retries >= 3: 
        print("Unable to process request, try again...")
        continue

    print("Code generated")
    print(cleaned_json["code"])
    print("\n\nDesciption:", cleaned_json["description"])

    filename = cleaned_json["filename"]

    try:
        with open(os.path.join("output", filename), "w") as f:
            f.write(cleaned_json["code"])
        print("Saved file", filename)
    except:
        print("Error saving file...")

> Running step 684e0773-36a4-48c6-94ae-ed81b3c2c73a. Step input: read the contents of test.py and write a python script that calls the post end point to make a new item.
Thought: The current language of the user is: Python. I need to use a tool to help me answer the question.
Action: code_reader
Action Input: {'file_name': 'test.py'}
Observation: {'file_content': 'from flask import Flask, request, jsonify\n\napp = Flask(__name__)\n\n# In-memory "database" for simplicity\nitems = []\n\n\n# Create\n@app.route("/items", methods=["POST"])\ndef create_item():\n    data = request.get_json()\n    items.append(data)\n    return jsonify(data), 201\n\n\n# Read all\n@app.route("/items", methods=["GET"])\ndef read_items():\n    return jsonify(items)\n\n\n# Read one\n@app.route("/items/<int:item_id>", methods=["GET"])\ndef read_item(item_id):\n    if item_id < 0 or item_id >= len(items):\n        return "Item not found.", 404\n    return jsonify(items[item_id])\n\n\n# Update\n@app.route("/items/<in

# Example from https://docs.llamaindex.ai/en/stable/examples/pipeline/query_pipeline/

In [6]:
class Movie(BaseModel):
    """Object representing a single movie."""

    name: str = Field(..., description="Name of the movie.")
    year: int = Field(..., description="Year of the movie.")


class Movies(BaseModel):
    """Object representing a list of movies."""

    movies: List[Movie] = Field(..., description="List of movies.")


llm = OpenAI(model="gpt-3.5-turbo")        # Local version of Ollama doesn't work.

output_parser = PydanticOutputParser(Movies)
json_prompt_str = """
Please generate related movies to {movie_name}. Output with the following JSON format: 
"""
json_prompt_str = output_parser.format(json_prompt_str)  

# add JSON spec to prompt template
json_prompt_tmpl = PromptTemplate(json_prompt_str)

p = QueryPipeline(chain=[json_prompt_tmpl, llm, output_parser], verbose=True)
output = p.run(movie_name="Toy Story")
output

> Running module 9cbbc324-8b56-43f6-810e-9f94261efa87 with input: 
movie_name: Toy Story

> Running module d71851e4-6fec-41fa-b2b9-6de22adde033 with input: 
messages: 
Please generate related movies to Toy Story. Output with the following JSON format: 



Here's a JSON schema to follow:
{"$defs": {"Movie": {"description": "Object representing a single movie.", "pro...

> Running module 3ee04f56-4525-401a-a42d-7ad508bac602 with input: 
input: assistant: {
  "movies": [
    {
      "name": "Finding Nemo",
      "year": 2003
    },
    {
      "name": "Cars",
      "year": 2006
    },
    {
      "name": "Up",
      "year": 2009
    },
    {...



Movies(movies=[Movie(name='Finding Nemo', year=2003), Movie(name='Cars', year=2006), Movie(name='Up', year=2009), Movie(name='Inside Out', year=2015)])